In [1]:
from utils import *
import tensorflow as tf
from keras import backend as K

2022-10-22 18:00:37.215161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 18:00:37.325131: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 18:00:37.805957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-22 18:00:37.806056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

Check if keras is using GPU

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

2022-10-22 18:00:38.328049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 18:00:38.385375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 18:00:38.401608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 18:00:38.401788: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



2022-10-22 18:00:38.792800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 18:00:38.793005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 18:00:38.793128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 18:00:38.793222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4029 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5
2022-10-22 18:00:38.794452: I tensorflow/stream_executor/cuda/cuda_gpu_executo

['/device:GPU:0']

Get tickers

In [3]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x])

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

Read dataset

In [4]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
# df = df[:-60]

Define training sets and data size

In [16]:
in_sets = [0,1]
out_sets = [1]
data_size = 1850

input_tickers = set()
output_tickers = set()
for in_set in in_sets:
    input_tickers = input_tickers.union(set(tickers_sets[in_set]))

for out_set in out_sets:
    output_tickers = output_tickers.union(set(tickers_sets[out_set]))

input_tickers = list(input_tickers)
output_tickers = list(output_tickers)


Process data

In [17]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=60, output_size=20, \
                        feature_range=(0,1), return_log=False)

493 14


Split dataset i training, validation and test

In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

Define hyperparameters

In [8]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (500, 0.2, True), 
        (625, 0.2, True),
        (400, 0.2, True),
        (150, 0.2, False)
    ],
    [
        (300, 0.2, True), 
        (400, 0.2, True),
        (400, 0.2, True),
        (300, 0.2, False)
    ],
    [
        (40, 0.2, True), 
        (40, 0.2, True),
        (40, 0.2, True),
        (40, 0.2, True),
        (40, 0.2, True),
        (75, 0.2, False)
    ],
    [
        (300, 0.15, True), 
        (300, 0.15, True),
        (300, 0.15, False)
    ],
    [
        (600, 0.15, True), 
        (600, 0.15, True),
        (600, 0.15, False)
    ]
]


Create Models

In [9]:
models = []
hyperparameters = range(5)
for i in hyperparameters:
    layers_info = layers_info_list[i]
    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

Train networks

In [10]:
history_list = []
epochs_list = [100,100,100,100,100]

set_name = 'set1/'
plot_path_dir = 'metrics/' + set_name
extra_info = ''

j=0
for i in hyperparameters:

    model = models[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/100


2022-10-22 18:00:47.849274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


39/39 [==============================] - 7s 67ms/step - loss: 0.2154 - accuracy: 0.0677 - mse: 0.0592 - val_loss: 0.1166 - val_accuracy: 0.1609 - val_mse: 0.0157
Epoch 2/100
39/39 [==============================] - 2s 44ms/step - loss: 0.1217 - accuracy: 0.1308 - mse: 0.0163 - val_loss: 0.0910 - val_accuracy: 0.1970 - val_mse: 0.0096
Epoch 3/100
39/39 [==============================] - 2s 44ms/step - loss: 0.1061 - accuracy: 0.1638 - mse: 0.0123 - val_loss: 0.0899 - val_accuracy: 0.2083 - val_mse: 0.0091
Epoch 4/100
39/39 [==============================] - 2s 44ms/step - loss: 0.1024 - accuracy: 0.1864 - mse: 0.0115 - val_loss: 0.0771 - val_accuracy: 0.2714 - val_mse: 0.0068
Epoch 5/100
39/39 [==============================] - 2s 44ms/step - loss: 0.0947 - accuracy: 0.2128 - mse: 0.0099 - val_loss: 0.0746 - val_accuracy: 0.2818 - val_mse: 0.0064
Epoch 6/100
39/39 [==============================] - 2s 46ms/step - loss: 0.0904 - accuracy: 0.2457 - mse: 0.0091 - val_loss: 0.0875 - val_acc

In [11]:
for model in models:
    y_predict = model.predict(x_val)
    mse_ = np.square(np.subtract(y_val,y_predict)).mean(axis=1)
    var_ = np.var(np.subtract(y_val,y_predict), axis=1)
    e_ = np.sum(np.mean(np.sqrt(np.add(mse_,var_)),axis=0))
    

9/9 [==============================] - 1s 19ms/step


In [12]:
scaler2, x2, y2, dic2 = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=60, output_size=20, \
                        feature_range=(0,1), return_log=True)

479 85


In [13]:
x2_train, y2_train, x2_val, y2_val, x2_test, y2_test = train_val_test_split(x,y)

In [14]:
models2 = []
hyperparameters = range(5)

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models2.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

In [15]:
history2_list = []
epochs_list = [100,100,100,100,100]

set_name = 'set1/'
plot_path_dir = 'metrics/' + set_name
extra_info = '_RETURN_LOG'

j=0
for i in hyperparameters:

    model = models2[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x2_train,
            y=y2_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x2_val, y2_val)
        )

        history2_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/100
39/39 [==============================] - 6s 75ms/step - loss: 0.2256 - accuracy: 0.0498 - mse: 0.0658 - val_loss: 0.1124 - val_accuracy: 0.1148 - val_mse: 0.0146
Epoch 2/100
39/39 [==============================] - 2s 46ms/step - loss: 0.1223 - accuracy: 0.1283 - mse: 0.0164 - val_loss: 0.0951 - val_accuracy: 0.1887 - val_mse: 0.0102
Epoch 3/100
39/39 [==============================] - 2s 46ms/step - loss: 0.1112 - accuracy: 0.1661 - mse: 0.0135 - val_loss: 0.0918 - val_accuracy: 0.2177 - val_mse: 0.0092
Epoch 4/100
39/39 [==============================] - 2s 46ms/step - loss: 0.0966 - accuracy: 0.2150 - mse: 0.0103 - val_loss: 0.0918 - val_accuracy: 0.1880 - val_mse: 0.0092
Epoch 5/100
39/39 [==============================] - 2s 46ms/step - loss: 0.0923 - accuracy: 0.2508 - mse: 0.0093 - val_loss: 0.0793 - val_accuracy: 0.2850 - val_mse: 0.0071
Epoch 6/100
39/39 [==============================] - 2s 44ms/step - loss: 0.0875 - accuracy: 0.2755 - mse: 0.0084 - val_loss: 0.07